# Wk 1 - Homework - KenwanCheung

Prompt: Use Python Regular Expressions to identify top-10 most frequent causes of failed food inspections in Chicago.  You can download the dataset here: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5

In [2]:
import nltk
import re
import pandas as pd
import numpy as np

In [3]:
# import csv
fi = pd.read_csv('../wk1/Food_Inspections.csv')

In [5]:
fi.head(10)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2130049,"JOE & THE JUICE ILLINOIS, LLC",JOE & THE JUICE,2564512.0,Restaurant,Risk 2 (Medium),10 E DELAWARE PL,CHICAGO,IL,60611.0,01/05/2018,License,Pass,NaN,41.899255,-87.627835,"(41.89925505559848, -87.62783463799146)"
1,2130022,BOO BAE TEA INC,BOO BAE TEA INC,2570290.0,NaN,Risk 2 (Medium),1013 W Webster AVE,CHICAGO,IL,60614.0,01/05/2018,License,Not Ready,NaN,41.921620,-87.654051,"(41.92161984057171, -87.65405058579164)"
2,2130018,FRESHII,FRESHII,2446395.0,Restaurant,Risk 1 (High),1166 W MADISON ST,CHICAGO,IL,60607.0,01/05/2018,Canvass,Fail,"9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY P...",41.881731,-87.656851,"(41.881731324473414, -87.65685079354886)"
3,2129964,LINCOLN PARK PRESCHOOL,LINCOLN PARK PRESCHOOL & KINDERGARTEN,2215624.0,Daycare (2 - 6 Years),Risk 1 (High),108 W GERMANIA PL,CHICAGO,IL,60610.0,01/04/2018,License,Pass w/ Conditions,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,41.910486,-87.631996,"(41.91048634702192, -87.63199583676088)"
4,2129963,ORIGINAL STEAM,ORIGINAL STEAM,2574892.0,NaN,Risk 1 (High),2428 S WALLACE AVE,CHICAGO,IL,60616.0,01/04/2018,License,Not Ready,NaN,41.848386,-87.641960,"(41.84838625123219, -87.64196007758322)"
5,2129953,JERK TACO MAN 1,JERK TACO MAN,2529063.0,Restaurant,Risk 1 (High),4001 W JACKSON BLVD,CHICAGO,IL,60624.0,01/04/2018,Complaint,Fail,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.876976,-87.725666,"(41.87697594883604, -87.72566593724505)"
6,2129949,"LA CEBOLLITA GRILL #2, INC.",LA CEBOLLITA GRILL,2184654.0,Restaurant,Risk 1 (High),1807 S ASHLAND AVE,CHICAGO,IL,60608.0,01/04/2018,Canvass,Fail,12. HAND WASHING FACILITIES: WITH SOAP AND SAN...,41.857561,-87.665989,"(41.857561185250134, -87.66598939271974)"
7,2129931,"CLIFTON GRILL,INC.",CLIFTON GRILL,2164148.0,Restaurant,Risk 1 (High),6410-12 N CLAREMONT AVE,CHICAGO,IL,60645.0,01/04/2018,Complaint,Pass,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.998121,-87.688822,"(41.99812088754938, -87.68882213326917)"
8,2129924,"BALU SMOOTHIES AND MOOR, LLC",LA FITNESS HEALTH CLUB,2559797.0,Restaurant,Risk 2 (Medium),55 E RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2018,License,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.884388,-87.625869,"(41.8843876187293, -87.62586884949191)"
9,2129920,KEDZIE DISCOUNT MART,KEDZIE DISCOUNT MART,2506863.0,Grocery Store,Risk 3 (Low),2410 S KEDZIE AVE,CHICAGO,IL,60623.0,01/04/2018,License Re-Inspection,Pass,2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...,41.847767,-87.705285,"(41.84776668182944, -87.70528457405833)"


In [6]:
np.unique(fi.Results)

array(['Business Not Located', 'Fail', 'No Entry', 'Not Ready',
       'Out of Business', 'Pass', 'Pass w/ Conditions'], dtype=object)

In [89]:
# segment into own df
fail = fi[fi["Results"] == "Fail"].reset_index()
fail = fail.Violations

In [91]:
# check
fail.head(10)

0    9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY P...
1    3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...
2    12. HAND WASHING FACILITIES: WITH SOAP AND SAN...
3    18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
4    41. PREMISES MAINTAINED FREE OF LITTER, UNNECE...
5    16. FOOD PROTECTED DURING STORAGE, PREPARATION...
6    33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...
7    2. FACILITIES TO MAINTAIN PROPER TEMPERATURE -...
8    13. NO EVIDENCE OF RODENT OR INSECT INFESTATIO...
9    18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...
Name: Violations, dtype: object

In [93]:
# reading longer strings of reasons
fail[0]

'9. WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRESSURE - Comments: OBSERVE NO HOT RUNNING WATER ON THE PREMISES; THAT INCLUDES EXPOSED HAND SINKS IN FRONT & REAR PREP AREAS, 3-COMPARTMENT SINK IN REAR PREP, AND HAND SINKS IN BOTH TOILET ROOMS. INSTRUCTED TO CONTACT PLUMMER TO HAVE HOT WATER RESTORED. CRITICAL VIOLATION 7-38-030. | 29. PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42-090 - Comments: PREVIOUS MINOR VIOLATION NOT CORRECTED FROM INSPECTION REPORT 1989320, DATED 2/17/2017. VIOLATION INCLUDES; #38; NO RUNNING HOT AND COLD WATER TO TOP LOADING SOFT SERVE MACHINE, INSTRUCTED TO PROVIDE,  \n \nVIOLATION STILL EXISTS. SERIOUS VIOLATION 7-42-090 | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: MUST DISCONTINUE USING MILK CRATES AS STORAGE RACKS THROUGHOUT FRONT AND REAR PREP AREAS, AND IN THE WALK IN COOLER. INSTALL CORRECT STORAGE RACKS. | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METH

## Parsing

It seems like we need to pull the section past the . and space, which seems to be the automated comment. 

We should parse that string until the hyphen "-"

In [1]:
fail.tail()

NameError: name 'fail' is not defined

In [332]:
# test pattern
reason_codes = pd.DataFrame(columns = ['reason'])
counter = 0 

for i in range(0,len(fail)):
    test_text = str(fail[i])
    m = re.findall(r"\d\.\s(.*?)\s-\sComments:",test_text)
    if len(m) > 0:
        for j in range(0,len(m)):
            n = re.findall(r"\|\s\d.\.\s(.*)",str(m[j]))
            if len(n) > 0:
                reason_codes.loc[counter] = n
                counter += 1
            else:
                reason_codes.loc[counter] = m[j]
                counter += 1
    else:
        reason_codes.loc[counter] = None
        counter += 1

In [328]:
reason_codes[0:5]

,reason
0,"WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRES..."
1,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
2,HAND WASHING FACILITIES: WITH SOAP AND SANITAR...
3,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...
4,"PREMISES MAINTAINED FREE OF LITTER, UNNECESSAR..."


In [329]:
# remove NAs
reason_codes_clean = reason_codes.dropna(axis=0, how='any')
reason_codes_clean[0:10]

,reason
0,"WATER SOURCE: SAFE, HOT & COLD UNDER CITY PRES..."
1,POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE R...
2,HAND WASHING FACILITIES: WITH SOAP AND SANITAR...
3,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...
4,"PREMISES MAINTAINED FREE OF LITTER, UNNECESSAR..."
5,"FOOD PROTECTED DURING STORAGE, PREPARATION, DI..."
6,FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS C...
7,FACILITIES TO MAINTAIN PROPER TEMPERATURE
8,"NO EVIDENCE OF RODENT OR INSECT INFESTATION, N..."
9,NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS...


# Now we have a cleaned list of just the reason codes. We should thus find the highest frequency on this list

Let's use nltk to get the highest frequency

In [330]:
reason_freq = reason_codes_clean.reason.value_counts()
reason_freq[0:10]

NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS    7541
FACILITIES TO MAINTAIN PROPER TEMPERATURE                                                                                                 2012
PREVIOUS MINOR VIOLATION(S) CORRECTED 7-42-090                                                                                            1621
FOOD PROTECTED DURING STORAGE, PREPARATION, DISPLAY, SERVICE AND TRANSPORTATION                                                           1352
ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, DESIGNED, AND MAINTAINED                                                                         1332
POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATURE REQUIREMENT DURING STORAGE, PREPARATION DISPLAY AND SERVICE                                  1206
FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED                                                          1050

# Most common results:

The most common reason codes are in the 10 above. The most common reason code above all else seems to be protection from rodents, which is not shocking!